In [13]:
import pandas as pd
import numpy as np

from numpy.polynomial import Polynomial
from numpy import poly1d, polyfit

from scipy import integrate

import  matplotlib.pyplot as plt

In [5]:
qx = pd.read_csv('life_tables.csv', sep=';', decimal=',')
qx = qx['AT_49_F']

In [82]:
def D_to_C(x, y, deg):
    
    x = np.array(x)
    y = np.array(y)
    
    f = polyfit(x, y, deg=deg)
    f = poly1d(f)
    
    return f

# Calculando  função contínua de tpx

In [100]:
def tpx_discreta(qx, t, x):
    px = 1 - np.array(qx)
    tpx = np.prod(px[x:x+t])
    return tpx

In [144]:
def tpx_continua(qx, age, deg):
    y = [tpx_discreta(qx, tempo, x=age) for tempo in range(len(qx))]
    tpx = D_to_C(qx.index, y, deg)
    
    return tpx

# Calculando a forca de mortalidade

In [105]:
def fmu(sobrevivencia, t):
    
    return -1/sobrevivencia(t) * sobrevivencia.deriv()(t)

In [107]:
fmu(sobrevivencia=survival, t=10)

0.025393390528516798

# Calculando o fator desconto

In [108]:
def fator(delta, val):
    return np.exp(-delta*val)

# Calculando Ax continuo

In [175]:
sobrevivencia_continua = tpx_continua(qx=qx, age=50, deg=10)

In [176]:
forca = lambda tempo: sobrevivencia_continua(tempo)

In [178]:
fds = lambda tempo: fator(.0009, tempo)

In [179]:
def A(tempo):
    return forca(tempo)*fds(tempo)*teste(tempo)

In [186]:
integrate.quad(A, a=0, b=np.infty)

<ipython-input-186-6f8b5630bd76>:1: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  integrate.quad(A, a=0, b=np.infty)


(6.382060449135197e+48, 1.0112625425921728e+49)